In [1]:
import plotly.graph_objects as go
import numpy as np
import plotly.graph_objects as go
import pandas as pd
import os
import re

In [2]:
freq = 4
# Initialize an empty DataFrame
df = pd.DataFrame()

# Regular expression pattern for extracting numbers
pattern = re.compile(r'qubits_(\d+)_circ_layers_(\d+)')

# Keywords to look for in the directory names

keywords = ["SimplifiedTwoDesign", "local", "periodic_layers_02"]

# Traverse the logs directory
for root, dirs, files in os.walk("logs"):
    # Check if all keywords are present in the root directory
    if all(keyword in root for keyword in keywords):
        for file in files:
            if file == "abs_coeffs_var.npy":
                # Extract path components
                path_parts = "/".join(root.split("/")[1:])  # Exclude 'logs/' part

                # Use regex to extract numbers for qubits and circuit layers
                match = pattern.search(path_parts)
                if match:
                    qubits = int(match.group(1))
                    circ_layers = int(match.group(2))

                    # Construct the full file path and read the .npy file
                    file_path = os.path.join(root, file)
                    
                    data = np.load(file_path)

                    if qubits*circ_layers < freq:
                        pass
                    else:
                        value = data[freq]

                        # Populate the DataFrame
                        if circ_layers not in df.index:
                            df.loc[circ_layers, qubits] = value
                        else:
                            df.at[circ_layers, qubits] = value

# Sort DataFrame by index and columns
df.sort_index(inplace=True)
df.sort_index(axis=1, inplace=True)

# Add index and column titles
df.index.name = 'circuit_layers'
df.columns.name = 'n_qubits'

print(df)


Empty DataFrame
Columns: []
Index: []


In [3]:
z = df.values
x = df.index.values  # circuit_layers
y = df.columns.values#.astype(str)  # n_qubits, converted to strings to match your DataFrame

fig = go.Figure(data=[go.Surface(z=z.T, x=x, y=y)])

fig.update_layout(
    title=f"var_c_{freq}",  # Using f-string to dynamically set title
    autosize=False,
    width=1000,
    height=1000,
    margin=dict(l=65, r=50, b=65, t=90),
    scene=dict(
        xaxis_title="circ Layers",  
        yaxis_title="n_qubits",  
        zaxis_title=f"var_c_{freq}",
        xaxis=dict(range=[min(x), max(x)]),  # setting x-axis range
        yaxis=dict(range=[min(y), max(y)]),  # setting y-axis range
        zaxis=dict(range=[min(z.flatten()), max(z.flatten())])  # setting z-axis range
     
    )
)

ValueError: min() arg is an empty sequence

In [4]:
freq = 20

# Initialize a subplot with 1 row and 3 columns
fig = make_subplots(rows=1, cols=3, specs=[[{'type': 'surface'}, {'type': 'surface'}, {'type': 'surface'}]])

ansatz_types = ["SimplifiedTwoDesign", "BasicEntangling", "StronglyEntangling"]
keywords = ["local", "periodic_layers_02"]

# Regular expression pattern for extracting numbers
pattern = re.compile(r'qubits_(\d+)_circ_layers_(\d+)')

for idx, ansatz_type in enumerate(ansatz_types, 1):
    # Initialize an empty DataFrame
    df = pd.DataFrame()

    # Traverse the logs directory
    for root, dirs, files in os.walk("logs"):
        # Check if all keywords and the current ansatz_type are present in the root directory
        if all(keyword in root for keyword in keywords + [ansatz_type]):
            for file in files:
                if file == "abs_coeffs_avg.npy":
                    # ... (The rest of your code remains unchanged)

    # Sort DataFrame by index and columns
    df.sort_index(inplace=True)
    df.sort_index(axis=1, inplace=True)

    z = df.values
    x = df.index.values
    y = df.columns.values

    # Add this data to the subplot in the ith column
    fig.add_trace(go.Surface(z=z.T, x=x, y=y), row=1, col=idx)
    fig.update_layout(title=f"{ansatz_type} c_{freq}")

# Update layout and axis labels
fig.update_layout(
    autosize=False,
    width=3000,
    height=1000,
    margin=dict(l=65, r=50, b=65, t=90)
)

fig.update_scenes(
    xaxis_title="circ Layers",
    yaxis_title="n_qubits",
    zaxis_title=f"c_{freq}"
)

fig.show()


IndentationError: expected an indented block after 'if' statement on line 21 (2480795442.py, line 25)

In [5]:
import os
import re
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# List of frequencies to explore
frequencies = list(range(0, 6))

# Initialize a subplot with variable number of rows and 3 columns
fig = make_subplots(rows=len(frequencies), cols=3, subplot_titles=["SimplifiedTwoDesign", "BasicEntangling", "StronglyEntangling"], specs=[[{'type': 'surface'}]*3]*len(frequencies))

ansatz_types = ["SimplifiedTwoDesign", "BasicEntangling", "StronglyEntangling"]
keywords = ["local", "periodic_layers_02"]

# Regular expression pattern for extracting numbers
pattern = re.compile(r'qubits_(\d+)_circ_layers_(\d+)')

scene_counter = 1
for row_idx, freq in enumerate(frequencies, 1):
    for col_idx, ansatz_type in enumerate(ansatz_types, 1):
        # Initialize an empty DataFrame
        df = pd.DataFrame()

        # Traverse the logs directory
        for root, dirs, files in os.walk("logs"):
            if all(keyword in root for keyword in keywords + [ansatz_type]):
                for file in files:
                    if file == "abs_coeffs_avg.npy":
                        path_parts = "/".join(root.split("/")[1:])
                        match = pattern.search(path_parts)
                        if match:
                            qubits = int(match.group(1))
                            circ_layers = int(match.group(2))
                            file_path = os.path.join(root, file)
                            data = np.load(file_path)
                            if qubits * circ_layers < freq:
                                pass
                            else:
                                value = data[freq]
                                if circ_layers not in df.index:
                                    df.loc[circ_layers, qubits] = value
                                else:
                                    df.at[circ_layers, qubits] = value

        # Sort DataFrame
        df.sort_index(inplace=True)
        df.sort_index(axis=1, inplace=True)

        z = df.values
        x = df.index.values
        y = df.columns.values

        # Add this data to the subplot in the specified row and column
        fig.add_trace(go.Surface(z=z.T, x=x, y=y, colorscale='Viridis'), row=row_idx, col=col_idx)

        fig.update_scenes(
            xaxis_title="circ Layers",
            yaxis_title="n_qubits",
            zaxis_title=f"c_{freq}",
            selector=dict(row=row_idx, col=col_idx)
        )

        scene_counter += 1

# Update general layout
fig.update_layout(
    autosize=False,
    width=3000,
    height=1000 * len(frequencies),
    margin=dict(l=65, r=50, b=65, t=90)
)

fig.show()
